### Data Ingestion made as a script to automate the process.
### Provide the data path with single or multiple json files and it will load and combine the 
### Files in single Python DataFrame

In [ ]:
import os
import sys
import getopt
import scipy.stats as stats
import pandas as pd
import numpy as np
import sqlite3
import json
from pandas.io.json import json_normalize
import string
import re


def ingest_stream_data(file_path):
    """
    load and combine the stream data
    """
    os.chdir(file_path)
    
    List_Files = os.listdir(file_path)
    
    
    df_streams = []
    df_temp = []
    
    for file in List_Files:
        
        #print(file)
        str = file_path + '\\' + file
        #print(str)
                        
        df_temp = pd.read_json(str)
        df_temp.rename(columns = {'price':'total_price'}, inplace = True)
        df_temp.rename(columns = {'TimesViewed':'times_viewed'}, inplace = True)
        df_temp.rename(columns = {'StreamID':'stream_id'}, inplace = True)
        
        
        
        print(df_temp)
        
        df_streams.append(df_temp)
                  
    
    df_streams = pd.concat(df_streams, sort=True)
        
    print(df_streams)
    
    df_streams['stream_id'] = df_streams['stream_id'].str.extract(r'(\d+)', expand=False)
    df_streams['stream_id'].astype(int)
    return(df_streams)


In [ ]:
df_main = ingest_stream_data('C:\\Nilabja\\Supportive Docs and Material\\Badge and Training\\AI Workflow Specialization\\capctone\\ai-workflow-capstone-master\\ai-workflow-capstone-master\\cs-train')
os.chdir('C:\\Nilabja\\Supportive Docs and Material\\Badge and Training\\AI Workflow Specialization\\capctone\\ai-workflow-capstone-master\\ai-workflow-capstone-master')

df_main.to_csv('file1.csv')

In [ ]:
df_main.describe()
df_main.isnull().sum()
df_main.info()
df_rev = df_main.groupby('country')['total_price'].sum().reset_index()
df_rev = df_rev.sort_values(by=['total_price'], ascending= False)
print(df_rev)

In [ ]:
df_reorder=df_main[['country', 'customer_id', 'invoice', 'total_price', 'stream_id', 'times_viewed', 'year', 'month', 'day']]
df_reorder 

In [ ]:
df_reorder['Date']=pd.to_datetime(df_reorder[['year','month','day']])
df_reorder


In [ ]:
df_reorder.groupby(df_reorder['Date'].dt.strftime('%B'))['total_price'].sum().sort_values()

In [ ]:
df_reorder['year_month'] = df_reorder['Date'].dt.strftime('%Y-%m')
df_reorder

In [ ]:
df_price = df_reorder.groupby(['country', 'year_month'])['total_price'].sum().reset_index()
#df_price = df_rev.sort_values(by=['total_price'], ascending= False)
print(df_price)

In [ ]:
df_C_AUS = df_price[df_price['country'] == 'Australia']
df_C_AUS

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np

#fig = plt.figure()
#ax = plt.axes()
plt.figure(figsize=(20,10))

plt.plot(df_C_AUS['year_month'], df_C_AUS['total_price'])
plt.title("Price Curve For Australia")
plt.xlabel("Month")
plt.ylabel("Price");

In [ ]:
#pd.to_datetime([f'{y}-{m}-01' for y, m in zip(df.year, df.Months)])

df_C_AUS_Sales = df_C_AUS[['year_month', 'total_price']]

df_C_AUS_Sales['year_month'] = df_C_AUS_Sales['year_month'].astype(str) + '-1'

#df_C_AUS_Sales = pd.to_datetime([f'{ym}-01' for ym in df_C_AUS.year_month])

#df_C_AUS_Sales.set_index('year_month', inplace=True)
#df_C_AUS_Sales.info()
#df_C_AUS_Sales = df_C_AUS_Sales.set_index(pd.DatetimeIndex(df_C_AUS_Sales['year_month']))

pd.to_datetime(df_C_AUS_Sales['year_month'], format='%Y-%m-%d')
df_C_AUS_Sales = df_C_AUS_Sales.set_index('year_month')
df_C_AUS_Sales.index = pd.to_datetime(df_C_AUS_Sales.index)
df_C_AUS_Sales

In [ ]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'

In [ ]:
from pylab import rcParams

rcParams['figure.figsize'] = 18, 8
decomposition = sm.tsa.seasonal_decompose(df_C_AUS_Sales['total_price'], model='additive', freq=3)
fig = decomposition.plot()
plt.show()
#resid = decomposition.resid
#seasonal = decomposition.seasonal
#trend = decomposition.trend
#plt.plot(resid)
#plt.plot(seasonal)
#plt.plot(trend)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np

#fig = plt.figure()
#ax = plt.axes()
plt.figure(figsize=(20,10))

plt.plot(df_C_AUS['year_month'], df_C_AUS['total_price'])
plt.title("Price Curve For Australia")
plt.xlabel("Month")
plt.ylabel("Price");

In [ ]:
# AR example
from statsmodels.tsa.ar_model import AR
from random import random
# contrived dataset
#data = [x + random() for x in range(1, 100)]
# fit model
model = AR(df_C_AUS_Sales)
model_fit = model.fit()
# make prediction
yhat = model_fit.predict(len(df_C_AUS_Sales), len(df_C_AUS_Sales))
print(yhat)


In [ ]:
# ARIMA example
from statsmodels.tsa.arima_model import ARIMA
from random import random
# contrived dataset
#data = [x + random() for x in range(1, 100)]
# fit model
model = ARIMA(df_C_AUS_Sales, order=(1, 1, 1))
model_fit = model.fit(disp=False)
# make prediction
yhat = model_fit.predict(len(df_C_AUS_Sales), len(df_C_AUS_Sales), typ='levels')
print(yhat)

In [ ]:
df_C_AUS_Sales.index

In [ ]:
df_C_AUS_Sales.plot()